In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import numpy.random as nr
import math
from sklearn.neural_network import MLPClassifier
import sklearn.model_selection as ms
from sklearn import preprocessing
import sklearn.decomposition as skde
import sklearn.metrics as sklm
import sklearn.datasets as skds
import graphviz 
from sklearn import tree
import itertools

%matplotlib inline

In [2]:
bikesData = pd.read_csv('BikesProcessed.csv')
Labels = np.array(bikesData['BikeBuyer'])
bikesData.drop(bikesData.columns[0], axis=1, inplace=True)
bikesData.drop(['BikeBuyer'], axis=1, inplace=True)
Features = np.array(bikesData)
print(Features.shape)
bikesData.head()

(16404, 12)


,HomeOwnerFlag,NumberCarsOwned,NumberChildrenAtHome,AreaCode,Sex,Married,JobType,EducationType,Country,Income,AgeBracket,ChildrenFlag
0,1,0,0,500,1,1,0,0,1,3,0,1
1,0,1,3,500,1,0,0,0,1,3,1,1
2,1,1,3,500,1,1,0,0,1,2,1,1
3,0,1,0,500,0,0,0,0,1,2,0,0
4,1,4,5,500,0,0,0,0,1,2,0,1


In [3]:
df_1 = pd.read_csv('Top_CV_DecisionTree.csv')
df_2 = pd.read_csv('Top_CV_NaiveBayes.csv')
df_3 = pd.read_csv('Top_CV_SVM.csv')

important_features = pd.concat([df_1,df_2, df_3])
important_features = important_features.groupby('combs').head(1)
print(important_features.shape)
important_features.head()

(114, 5)


,Unnamed: 0,Unnamed: 0.1,size,accuracy,combs
0,2388,2388,6,0.793000,"('NumberChildrenAtHome', 'Sex', 'Married', 'In..."
1,3242,3242,7,0.792400,"('NumberChildrenAtHome', 'Sex', 'Married', 'Jo..."
2,1403,1403,5,0.792375,"('NumberChildrenAtHome', 'Sex', 'Married', 'Ag..."
3,2313,2313,6,0.792125,"('NumberChildrenAtHome', 'AreaCode', 'Sex', 'M..."
4,1401,1401,5,0.792125,"('NumberChildrenAtHome', 'Sex', 'Married', 'In..."


In [4]:
combs = np.array(important_features['combs'])
feature_combs = []
for item in combs:
    item = item.replace('(', '')
    item = item.replace(')', '')
    item = item.replace(',', '')
    item = item.replace('\'', '')
    item = item.split()
    feature_combs.append(item)
feature_combs

[['NumberChildrenAtHome',
  'Sex',
  'Married',
  'Income',
  'AgeBracket',
  'ChildrenFlag'],
 ['NumberChildrenAtHome',
  'Sex',
  'Married',
  'JobType',
  'EducationType',
  'AgeBracket',
  'ChildrenFlag'],
 ['NumberChildrenAtHome', 'Sex', 'Married', 'AgeBracket', 'ChildrenFlag'],
 ['NumberChildrenAtHome',
  'AreaCode',
  'Sex',
  'Married',
  'AgeBracket',
  'ChildrenFlag'],
 ['NumberChildrenAtHome', 'Sex', 'Married', 'Income', 'AgeBracket'],
 ['HomeOwnerFlag',
  'NumberChildrenAtHome',
  'Sex',
  'Married',
  'AgeBracket',
  'ChildrenFlag'],
 ['NumberChildrenAtHome',
  'Sex',
  'Married',
  'EducationType',
  'AgeBracket',
  'ChildrenFlag'],
 ['NumberChildrenAtHome',
  'AreaCode',
  'Sex',
  'Married',
  'JobType',
  'EducationType',
  'AgeBracket',
  'ChildrenFlag'],
 ['NumberChildrenAtHome', 'Sex', 'Married', 'JobType', 'AgeBracket'],
 ['HomeOwnerFlag',
  'NumberChildrenAtHome',
  'Sex',
  'Married',
  'Income',
  'AgeBracket',
  'ChildrenFlag'],
 ['NumberCarsOwned',
  'NumberCh

In [5]:
#Scale the data
scaler = preprocessing.StandardScaler().fit(Features)
Features = scaler.transform(Features)
titles = list(bikesData)
bikes_scaled = pd.DataFrame()
count = 0
Features = pd.DataFrame(Features)

for title in titles:
    bikes_scaled[title] = Features[count]
    count+=1

bikes_scaled.head()

/Users/grahamcooper/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/Users/grahamcooper/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


,HomeOwnerFlag,NumberCarsOwned,NumberChildrenAtHome,AreaCode,Sex,Married,JobType,EducationType,Country,Income,AgeBracket,ChildrenFlag
0,0.695306,-1.316615,-0.654523,-1.024189,0.989328,0.923656,-1.128274,-1.148595,-0.296932,1.246941,-1.011425,0.590773
1,-1.438215,-0.438729,1.326679,-1.024189,0.989328,-1.082654,-1.128274,-1.148595,-0.296932,1.246941,0.288843,0.590773
2,0.695306,-0.438729,1.326679,-1.024189,0.989328,0.923656,-1.128274,-1.148595,-0.296932,0.398228,0.288843,0.590773
3,-1.438215,-0.438729,-0.654523,-1.024189,-1.010787,-1.082654,-1.128274,-1.148595,-0.296932,0.398228,-1.011425,-1.692697
4,0.695306,2.194929,2.647481,-1.024189,-1.010787,-1.082654,-1.128274,-1.148595,-0.296932,0.398228,-1.011425,0.590773


In [6]:
def score_model(probs, threshold):
    return np.array([1 if x > threshold else 0 for x in probs[:,1]])

In [7]:
#cv = ms.ShuffleSplit(n_splits=8, test_size=5000, random_state=0)
Labels = Labels.reshape(Labels.shape[0],)
#scoring = ['accuracy','precision_macro', 'recall_macro', 'f1_macro', 'roc_auc']

In [8]:
size = []
accuracy = []
#f1 = []
#precision = []
#recall = []
#auc = []
est_hidden_layer_sizes = []
cv_mean = []
cv_std = []
count = 0
for comb in feature_combs:
    comb_features = np.array(bikes_scaled[np.array(comb)])

    print(comb)
    v1 = len(comb)
    v2 = math.ceil(len(comb)*1.5)+1
    v3 = int(len(comb)*0.75)
    v4 = math.ceil(len(comb)*1.25)+1
    
    #structure = list({(v1, v3), (v1, v1), (v1, v4), (v2, v3), (v2, v1), (v2, v4), (v4, v3), (v4, v1), (v4, v4)})
    #print(structure)
    
    structure = set()
    for i in range(v1, v2):
        for j in range(v3, v4):
            if i>=j and j>0:
                structure.add((i,j))
                
    structure = list(structure)
    print(structure)
    
    nr.seed(123)
    inside = ms.KFold(n_splits=8, shuffle = True)
    nr.seed(321)
    outside = ms.KFold(n_splits=5, shuffle = True)
    
    ## Define the dictionary for the grid search and the model object to search on
    param_grid = {#"alpha":[0.0000001,0.000001,0.00001], 
              #"early_stopping":[True, False], 
              #"beta_1":[0.95,0.90,0.80], 
              #"beta_2":[0.999,0.9,0.8],
                "hidden_layer_sizes": structure}
    
    ## Define the random forest model
    nr.seed(3456)
    rf_clf = MLPClassifier()#max_iter = 300

    ## Perform the grid search over the parameters
    nr.seed(4455)
    rf_clf = ms.GridSearchCV(estimator = rf_clf, param_grid = param_grid, 
                          cv = inside, # Use the inside folds
                          scoring = 'accuracy', #scoring,
                          return_train_score = True)
    rf_clf.fit(comb_features, Labels)
    est_hidden_layer_sizes.append(rf_clf.best_estimator_.hidden_layer_sizes)
    
    
    nr.seed(498)
    cv_estimate = ms.cross_val_score(rf_clf, comb_features, Labels, 
                                 cv = outside) # Use the outside folds
    cv_mean.append(np.mean(cv_estimate))
    cv_std.append(np.std(cv_estimate))

    # Randomly sample cases to create independent training and test data

    nr.seed(1115)
    indx = range(comb_features.shape[0])
    indx = ms.train_test_split(indx, test_size = 5000)
    X_train = comb_features[indx[0],:]
    y_train = np.ravel(Labels[indx[0]])
    X_test = comb_features[indx[1],:]
    y_test = np.ravel(Labels[indx[1]])
    
    nr.seed(1115)
    rf_mod = MLPClassifier(hidden_layer_sizes = rf_clf.best_estimator_.hidden_layer_sizes)
                   #alpha = nn_clf.best_estimator_.alpha, 
                   #early_stopping = nn_clf.best_estimator_.early_stopping, 
                   #beta_1 = nn_clf.best_estimator_.beta_1, 
                   #beta_2 = nn_clf.best_estimator_.beta_2,
                   #max_iter = 300)
    rf_mod.fit(X_train, y_train)
    
    probabilities = rf_mod.predict_proba(X_test)
    scores = score_model(probabilities, 0.75)
    
    accuracy.append(sklm.accuracy_score(y_test, scores))
    #f1.append(np.mean(scores['test_f1_macro']))
    #precision.append(np.mean(scores['test_precision_macro']))
    #recall.append(np.mean(scores['test_recall_macro']))
    #auc.append(np.mean(scores['test_roc_auc']))
    size.append(len(comb))
    count += 1
    print(round(count/len(feature_combs), 2))

['NumberChildrenAtHome', 'Sex', 'Married', 'Income', 'AgeBracket', 'ChildrenFlag']
[(6, 4), (9, 7), (6, 6), (7, 6), (9, 8), (7, 7), (9, 5), (7, 4), (7, 5), (8, 8), (8, 7), (9, 4), (9, 6), (8, 6), (8, 5), (6, 5), (8, 4)]
0.01
['NumberChildrenAtHome', 'Sex', 'Married', 'JobType', 'EducationType', 'AgeBracket', 'ChildrenFlag']
[(9, 8), (10, 6), (7, 7), (8, 5), (10, 8), (11, 5), (10, 7), (7, 6), (8, 6), (10, 9), (9, 7), (11, 9), (7, 5), (8, 7), (9, 6), (11, 7), (10, 5), (11, 8), (9, 9), (8, 8), (9, 5), (11, 6)]
0.02
['NumberChildrenAtHome', 'Sex', 'Married', 'AgeBracket', 'ChildrenFlag']
[(6, 4), (5, 4), (7, 3), (8, 3), (5, 5), (6, 6), (7, 6), (7, 7), (6, 3), (7, 4), (7, 5), (8, 7), (8, 4), (8, 6), (8, 5), (6, 5), (5, 3)]
0.03
['NumberChildrenAtHome', 'AreaCode', 'Sex', 'Married', 'AgeBracket', 'ChildrenFlag']
[(6, 4), (9, 7), (6, 6), (7, 6), (9, 8), (7, 7), (9, 5), (7, 4), (7, 5), (8, 8), (8, 7), (9, 4), (9, 6), (8, 6), (8, 5), (6, 5), (8, 4)]
0.04
['NumberChildrenAtHome', 'Sex', 'Married

/Users/grahamcooper/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


0.27
['HomeOwnerFlag', 'NumberChildrenAtHome', 'Sex', 'Married']
[(6, 4), (5, 4), (5, 5), (4, 4), (6, 3), (4, 3), (6, 5), (5, 3)]
0.28
['NumberChildrenAtHome', 'AreaCode', 'Sex', 'Married', 'JobType', 'EducationType', 'Country', 'AgeBracket', 'ChildrenFlag']
[(11, 11), (9, 8), (10, 6), (12, 12), (13, 8), (14, 9), (12, 9), (10, 8), (11, 10), (13, 12), (10, 7), (12, 6), (13, 7), (12, 10), (10, 9), (9, 7), (13, 11), (14, 10), (12, 7), (11, 9), (14, 6), (13, 6), (12, 11), (9, 6), (13, 10), (14, 11), (11, 7), (10, 10), (11, 8), (14, 7), (9, 9), (14, 12), (13, 9), (14, 8), (11, 6), (12, 8)]
0.29
['NumberCarsOwned', 'NumberChildrenAtHome', 'AreaCode', 'Sex', 'Married', 'JobType', 'EducationType', 'AgeBracket', 'ChildrenFlag']
[(11, 11), (9, 8), (10, 6), (12, 12), (13, 8), (14, 9), (12, 9), (10, 8), (11, 10), (13, 12), (10, 7), (12, 6), (13, 7), (12, 10), (10, 9), (9, 7), (13, 11), (14, 10), (12, 7), (11, 9), (14, 6), (13, 6), (12, 11), (9, 6), (13, 10), (14, 11), (11, 7), (10, 10), (11, 8), (

0.52
['HomeOwnerFlag', 'NumberCarsOwned', 'NumberChildrenAtHome', 'AreaCode', 'Sex', 'Married', 'EducationType', 'AgeBracket', 'ChildrenFlag']
[(11, 11), (9, 8), (10, 6), (12, 12), (13, 8), (14, 9), (12, 9), (10, 8), (11, 10), (13, 12), (10, 7), (12, 6), (13, 7), (12, 10), (10, 9), (9, 7), (13, 11), (14, 10), (12, 7), (11, 9), (14, 6), (13, 6), (12, 11), (9, 6), (13, 10), (14, 11), (11, 7), (10, 10), (11, 8), (14, 7), (9, 9), (14, 12), (13, 9), (14, 8), (11, 6), (12, 8)]
0.53
['NumberChildrenAtHome', 'AreaCode', 'Sex', 'Married', 'EducationType', 'Country', 'AgeBracket', 'ChildrenFlag']
[(9, 8), (10, 6), (12, 9), (10, 8), (11, 10), (10, 7), (12, 6), (8, 6), (12, 10), (10, 9), (9, 7), (12, 7), (11, 9), (8, 7), (9, 6), (11, 7), (10, 10), (11, 8), (9, 9), (8, 8), (11, 6), (12, 8)]
0.54
['NumberCarsOwned', 'NumberChildrenAtHome', 'AreaCode', 'Sex', 'Married', 'EducationType', 'Country', 'AgeBracket', 'ChildrenFlag']
[(11, 11), (9, 8), (10, 6), (12, 12), (13, 8), (14, 9), (12, 9), (10, 8), 

0.75
['NumberChildrenAtHome', 'Sex', 'AgeBracket']
[(3, 2), (5, 4), (3, 3), (4, 4), (4, 3), (4, 2), (5, 2), (5, 3)]
0.75
['NumberChildrenAtHome', 'AreaCode', 'Married']
[(3, 2), (5, 4), (3, 3), (4, 4), (4, 3), (4, 2), (5, 2), (5, 3)]
0.76
['NumberChildrenAtHome', 'Married', 'ChildrenFlag']
[(3, 2), (5, 4), (3, 3), (4, 4), (4, 3), (4, 2), (5, 2), (5, 3)]
0.77
['NumberChildrenAtHome', 'Income']
[(3, 2), (3, 3), (3, 1), (2, 1), (2, 2)]
0.78
['JobType']
[(1, 1), (2, 1), (2, 2)]
0.79
['Married']
[(1, 1), (2, 1), (2, 2)]
0.8
['NumberCarsOwned', 'NumberChildrenAtHome', 'Sex', 'Married', 'JobType', 'EducationType', 'Income', 'AgeBracket', 'ChildrenFlag']
[(11, 11), (9, 8), (10, 6), (12, 12), (13, 8), (14, 9), (12, 9), (10, 8), (11, 10), (13, 12), (10, 7), (12, 6), (13, 7), (12, 10), (10, 9), (9, 7), (13, 11), (14, 10), (12, 7), (11, 9), (14, 6), (13, 6), (12, 11), (9, 6), (13, 10), (14, 11), (11, 7), (10, 10), (11, 8), (14, 7), (9, 9), (14, 12), (13, 9), (14, 8), (11, 6), (12, 8)]
0.81
['Numbe

0.96
['HomeOwnerFlag', 'NumberChildrenAtHome', 'Sex', 'Married', 'JobType', 'AgeBracket']
[(6, 4), (9, 7), (6, 6), (7, 6), (9, 8), (7, 7), (9, 5), (7, 4), (7, 5), (8, 8), (8, 7), (9, 4), (9, 6), (8, 6), (8, 5), (6, 5), (8, 4)]
0.97
['NumberChildrenAtHome', 'Married', 'JobType', 'AgeBracket']
[(6, 4), (5, 4), (5, 5), (4, 4), (6, 3), (4, 3), (6, 5), (5, 3)]
0.98
['NumberChildrenAtHome', 'JobType']
[(3, 2), (3, 3), (3, 1), (2, 1), (2, 2)]
0.99
['EducationType']
[(1, 1), (2, 1), (2, 2)]
1.0


In [10]:
midx = pd.DataFrame()
midx['est_hidden_layer_sizes'] = est_hidden_layer_sizes
midx['cv_mean'] = cv_mean
midx['cv_std'] = cv_std
midx['size'] = size
#midx['CV_accuracy'] = important_features['accuracy']
midx['accuracy'] = accuracy
#midx['precision'] = precision
#midx['recall'] = recall
#midx['f1'] = f1
#midx['AUC'] = auc
midx['combs'] = feature_combs
midx.to_csv('NCV_NuralNetworks_Structure.csv')